# Volur Client SDK Reference Guide

---

## Content
- [Installation](#installation)
- [Configuration](#configuration)
    - [VolurClient](#volurclient)
    - [Authentication](#authentication)
- [Data structures](#data-structures)
    - [Column](#column)
    - [QuanityColumn](#quantitycolumn)
    - [CharacteristicColumn](#characteristiccolumn)
    - [Value](#value)
- [Sources](#sources)
    - [MaterialsCSVFileSource](#materialscsvfilesource)
- [Methods](#methods)
    - [upload_materials](#upload_materials)

## Installation
todo

## Configuration

### VolurClient

```python
class VolurClient:
        self,
        auth_token: str | None = None,
```
Provides methods for uploading data to Volur.

**Parameters**\
`auth_token`&nbsp;&nbsp;&nbsp;&nbsp;Authentication token.

**Exceptions**\
`ValidationError`&nbsp;&nbsp;&nbsp;&nbsp;Raised if no token has been set in the `VOLUR_AUTH_TOKEN` variable or passed through the `auth_token` parameter. 

### Authentication
You can either create a `.env` file and set the `VOLUR_AUTH_TOKEN` variable to 
your access token or pass the token directly to the client using the
`auth_token` parameter.

Running the cell below will yield a `ValidationError` as explained above. 
Setting the `VOLUR_AUTH_TOKEN` in your `.env` file or passing your auth token
in the `auth_token` parameter in the second line of code should successfully
initialize the client.

In [ ]:
from volur.sdk.client import VolurClient

# Initialize the client (with auth token set in .env)
# Will throw an error if no variable is set in your .env file
client = VolurClient()
# or by setting the token directly
client = VolurClient(auth_token="your_auth_token")

## Data structures

Several of the methods and classes in the SDK utilize different data structures 
either on top or as input. These data structures are defined in the table under,
and more details can be found in the link to each separate structure.

| Source | Description |
| ----------- | ----------- |
| [Column](#column) | Class which defines a simple column. |
| [QuantityColumn](#quantitycolumn) | Class which defines a column with a value that represents a quantity. |
| [CharacteristicColumn](#characteristiccolumn) | Class which defines a column with a value that represents a given characteristic. |
| [Value](#value) | Class which defines the value of a characteristic given a specific data type. |

### Column

```python
class Column(BaseModel):
    column_name: str
```

Class which defines a simple column.

**Fields**\
`column_name`&nbsp;&nbsp;&nbsp;&nbsp;The name of the column


### QuantityColumn

```python
class QuantityColumn(Column):
    unit: Literal[
        "kilogram",
        "pound",
        "box",
        "piece",
    ]
```

Class which defines a column with a value that represents a quantity.

**Fields**\
`unit`&nbsp;&nbsp;&nbsp;&nbsp;The unit the quanity is given in. Can be either kilogram, pound, box, or piece.

### CharacteristicColumn

```python
class CharacteristicColumn(Column):
    characteristic_name: str
    data_type: Literal[
        "string",
        "bool",
        "integer",
        "float",
        "datetime",
        "date",
    ]
```

Class which defines a column with a value that represents a given characteristic.

**Fields**\
`characteristic_name`&nbsp;&nbsp;&nbsp;&nbsp;The unit the quanity is given in. Can be either kilogram, pound, box, or piece.\
`data_type`&nbsp;&nbsp;&nbsp;&nbsp;The data type the characteristic is given in. Can be either string, bool, integer, float, datetime, or date.

### Value

```python
class Value(BaseModel):
    value_string: str | None
    value_integer: int | None
    value_float: float | None
    value_bool: bool | None
```

Class which defines the value of a characteristic given a specific data type.

**Fields**\
`value_string`&nbsp;&nbsp;&nbsp;&nbsp;The string value of the characteristic.\
`value_integer`&nbsp;&nbsp;&nbsp;&nbsp;The int value of the characteristic.\
`value_float`&nbsp;&nbsp;&nbsp;&nbsp;The float value of the characteristic.\
`value_bool`&nbsp;&nbsp;&nbsp;&nbsp;The bool value of the characteristic.

## Sources

When uploading data, each method expects a data-specific source object as input.
These objects allow you to specify a mapping from your data to the format
in which the API expects to receive it, in addition to specifying which data 
type each column consists of.

| Source | Description |
| ----------- | ----------- |
| [MaterialsCSVFileSource](#MaterialsCSVFileSource) | Class which defines a schema when uploading materials data with a CSV file as source. |

### MaterialsCSVFileSource

```python
class MaterialsCSVFileSource(
        path: str,
        material_id_column: Column,
        plant_id_column: Column,
        quantity_column: QuantityColumn,
        characteristics_columns: CharacteristicColumn[],
    )
```

Here will see an example of how to upload data which is in a CSV file. 
source.

**Parameters**\
`path`&nbsp;&nbsp;&nbsp;&nbsp;Path to CSV file with materials data.\
`material_id_column`&nbsp;&nbsp;&nbsp;&nbsp;The column name used for getting the material identifier in the Material entity.\
`plant_id_column`&nbsp;&nbsp;&nbsp;&nbsp;The column name used for getting the plant identifier in the Material entity.\
`quantity_column`&nbsp;&nbsp;&nbsp;&nbsp;The column name used for getting the quantity in the Material entity.\
`characteristics_columns`&nbsp;&nbsp;&nbsp;&nbsp;The column names used for getting the material characteristics in the Material entity.

**Exceptions**\
`ValueError`&nbsp;&nbsp;&nbsp;&nbsp;This error is raised when there is an error pertaining to a value. The correlating output should explain what is the source of the error, such as a value missing, an unknown type being passed, columns or paths not existing, and more.

### MaterialsCSVFileSource example

In [ ]:
from volur.sdk.sources.csv import (
    CharacteristicColumn,
    Column,
    MaterialsCSVFileSource,
    QuantityColumn,
)

# Define a source for materials, with a single characteristic
source = MaterialsCSVFileSource(
    path="./data/materials.csv",
    material_id_column=Column(column_name="id"),
    plant_id_column=Column(column_name="plant"),
    quantity_column=QuantityColumn(column_name="quantity", unit="kilogram"),
    characteristics_columns=[
        CharacteristicColumn(
            column_name="char_value",
            characteristic_name="char_value",
            data_type="integer",
        )
    ],
)

## Methods

| Method | Description |
| ----------- | ----------- |
| [upload_materials](#upload_materials) | Method for uploading materials data. |

### upload_materials

```python
def upload_materials(
        self,
        source: MaterialSource
    )
```

Method for uploading materials data.

**Parameters**\
`source`&nbsp;&nbsp;&nbsp;&nbsp;The source of the data.

**Returns**\
A dictionary with a status and responses. The status will inform about how many
materials were uploaded, or if an error occurred and no materials were uploaded.
The response element will contain the response from the server. 

### upload_materials example

In [ ]:
# Using the MaterialsCSVFileSource defined above, upload the materials to the
# Volur platform.
client.upload_materials(source)